In [10]:
import base64
from io import BytesIO

from PIL import Image


def embeddable_image(data):
    img_data = 255 - 15 * data.astype(np.uint8)
    image = Image.fromarray(img_data, mode='L').resize((64, 64), Image.BICUBIC)
    buffer = BytesIO()
    image.save(buffer, format='png')
    for_encoding = buffer.getvalue()
    return 'data:image/png;base64,' + base64.b64encode(for_encoding).decode()

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

# output_notebook()

In [13]:
from sklearn.datasets import load_digits
from viskit.embedding.ivhd import Ivhd

import pandas as pd

digits = load_digits()

mapper = Ivhd(
    optimizer="force-directed",
    n_iter=3000,
    nn=10,
    rn=1,
    l1_steps=0,
    reverse_neighbors_steps=0,
    reverse_neighbors_count=0,
)

# mapper = UMAP()
embedding, neighbors = mapper.fit_transform(pd.DataFrame(digits.data), pd.DataFrame(digits.target))

Generating kNN graph...
Searching...
Finished.
b'[Info] [CSV Parser] Loading dataset from file: /Users/bartoszminch/Documents/Repositories/viskit/python/viskit/input_data.csv'
b'[Info] [CSV Parser] Loading dataset from file: /Users/bartoszminch/Documents/Repositories/viskit/python/viskit/input_labels.csv'
b'[Info] [CSV Parser] Labels file passed.'
b'[Info] [CSV Parser] Dataset size: 1797'
b'[Info] [CSV Parser] Dataset dimensionality: 64'
b'[Info] [CSV Parser] Number of classes in dataset: 10'
b'[Info] [CSV Parser] Finished.'
b'[Info] [Graph] Loading graph from cache...'
b'[Info] [Graph] Finished.'
b'[Info] [kNN Generator] Determining random neighbors...'


0:00:00.006499


b'[Info] [kNN Generator] Finished.'
b'[Info] [kNN Generator] Neighbors in graph: 19767'
b'Step: 0'
b'Step: 100'
b'Step: 200'
b'Step: 300'
b'Step: 400'
b'Step: 500'
b'Step: 600'
b'Step: 700'
b'Step: 800'
b'Step: 900'
b'Step: 1000'
b'Step: 1100'
b'Step: 1200'
b'Step: 1300'
b'Step: 1400'
b'Step: 1500'
b'Step: 1600'
b'Step: 1700'
b'Step: 1800'
b'Step: 1900'
b'Step: 2000'
b'Step: 2100'
b'Step: 2200'
b'Step: 2300'
b'Step: 2400'
b'Step: 2500'
b'Step: 2600'
b'Step: 2700'
b'Step: 2800'
b'Step: 2900'


In [14]:
import numpy as np

digits_df = pd.DataFrame(embedding, columns=('x', 'y'))
digits_df['digit'] = [str(x) for x in digits.target]
digits_df['image'] = list(map(embeddable_image, digits.images))
digits_df['nn'] = [str(x) for x in neighbors[:,0]]
digits_df['rn'] = [str(x) for x in neighbors[:,1]]

datasource = ColumnDataSource(digits_df)
color_mapping = CategoricalColorMapper(factors=[str(9 - x) for x in digits.target_names],
                                       palette=Spectral10)

plot_figure = figure(
    title='IVHD projection of the Digits dataset',
    plot_width=900,
    plot_height=900,
    tools='pan, wheel_zoom, reset'
)

plot_figure.add_tools(HoverTool(tooltips="""
<div>
    <div>
        <img src='@image' style='float: left; margin: 5px 5px 5px 5px'/>
    </div>
    <div>
        <span style='font-size: 16px; color: #224499'><p>Digit: @digit</p></span>
        <span style='font-size: 16px; color: #224499'><p>NN: @nn</p></span>
        <span style='font-size: 16px; color: #224499'><p>RN: @rn</p></span>
    </div>
</div>
"""))

plot_figure.circle(
    'x',
    'y',
    source=datasource,
    color=dict(field='digit', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4
)
show(plot_figure)